# 4) Functions Challenge — Capstone Exercises

**Goals:** design small, reusable functions; pass functions around; safe defaults; error handling; docstrings & tests; gluing pieces into a mini pipeline.

You can solve these by composing helpers you wrote earlier.

### A) Mini ETL: transactions → daily totals

Input: CSV lines like:

```
2025-08-01,alice,+10.50
2025-08-01,alice,-2
2025-08-02,bob,+5
bad,line
```

Produce: `{"2025-08-01": {"alice": 8.5}, "2025-08-02": {"bob": 5.0}}`

```python
def parse_line(line):
    """
    Return (date, user, amount_float) or None if invalid.
    Amount may have '+' or '-' sign; spaces allowed.
    """
    ...
def accumulate(lines):
    """
    Use parse_line; return nested dict date->{user: total}.
    """
    ...
lines = ["2025-08-01,alice,+10.50","2025-08-01,alice,-2","2025-08-02,bob,+5","bad,line"]
out = accumulate(lines)
assert out == {"2025-08-01":{"alice":8.5}, "2025-08-02":{"bob":5.0}}
```

### B) Validation + normalization + mapping

```python
def normalize_user(name, email):
    """
    Return dict {'name': 'Title Case', 'email': lower} or raise ValueError.
    """
    ...
def map_users(rows, *, validator=normalize_user):
    """
    rows: list of (name,email). Use validator to normalize each; skip invalid.
    """
    ...
rows = [(" ada lovelace ","ADA@EXAMPLE.COM"), ("bad","noats")]
ok = map_users(rows)
assert ok == [{"name":"Ada Lovelace","email":"ada@example.com"}]
```


In [3]:
# ========== A) Mini ETL: transactions → daily totals ==========

import re

_num_re = re.compile(r'^[+-]?\d+(?:\.\d+)?$')

def parse_line(line):
    """
    Return (date, user, amount_float) or None if invalid.
    Amount may have '+' or '-' sign; spaces allowed.
    """
    if not isinstance(line, str):
        return None
    parts = line.split(",")
    if len(parts) != 3:
        return None
    date, user, amount = (p.strip() for p in parts)
    if not date or not user:
        return None
    amt_str = amount.replace(",", "")  # tolerate thousands separators if present
    if not _num_re.match(amt_str):
        return None
    try:
        amt = float(amt_str)
    except ValueError:
        return None
    return (date, user, amt)

def accumulate(lines):
    """
    Use parse_line; return nested dict date->{user: total}.
    """
    out = {}
    for line in lines:
        parsed = parse_line(line)
        if not parsed:
            continue
        date, user, amt = parsed
        if date not in out:
            out[date] = {}
        out[date][user] = out[date].get(user, 0.0) + amt
    return out

# Quick test
lines = ["2025-08-01,alice,+10.50",
         "2025-08-01,alice,-2",
         "2025-08-02,bob,+5",
         "bad,line"]
out = accumulate(lines)
assert out == {"2025-08-01":{"alice":8.5}, "2025-08-02":{"bob":5.0}}

In [4]:
# ========== B) Validation + normalization + mapping ==========

def normalize_user(name, email):
    """
    Return dict {'name': 'Title Case', 'email': lower} or raise ValueError.
    """
    if not isinstance(name, str) or not isinstance(email, str):
        raise ValueError("name/email must be strings")
    name_norm = " ".join(part.capitalize() for part in name.strip().split())
    email_norm = email.strip().lower()
    if not name_norm:
        raise ValueError("name empty")
    if "@" not in email_norm:
        raise ValueError("invalid email")
    local, _, domain = email_norm.partition("@")
    if not local or "." not in domain or domain.startswith(".") or domain.endswith("."):
        raise ValueError("invalid email")
    return {"name": name_norm, "email": email_norm}

def map_users(rows, *, validator=normalize_user):
    """
    rows: list of (name,email). Use validator to normalize each; skip invalid.
    """
    out = []
    for name, email in rows:
        try:
            out.append(validator(name, email))
        except Exception:
            # skip invalid rows
            pass
    return out

# Quick test
rows = [(" ada lovelace ","ADA@EXAMPLE.COM"), ("bad","noats")]
ok = map_users(rows)
assert ok == [{"name":"Ada Lovelace","email":"ada@example.com"}]